In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import string
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import re
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [4]:
pip install gensim

     |████████████████████████████████| 24.2 MB 19.7 MB/s 
     |████████████████████████████████| 111 kB 43.4 MB/s 
     |████████████████████████████████| 30.7 MB 30.1 MB/s 
Note: you may need to restart the kernel to use updated packages.


In [3]:
from collections import OrderedDict
import spacy
from spacy import displacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from spacy.matcher import Matcher
from gensim.test.utils import datapath
from gensim.models.word2vec import Text8Corpus
from gensim.models.phrases import Phrases, Phraser
nlp = spacy.load('en_core_web_sm')

ModuleNotFoundError: No module named 'gensim'

In [3]:
def custom_retokenizer(doc):
    matcher = Matcher(nlp.vocab)
    patterns = [[{"LOWER": "el"}, {"LOWER": "paso"}]]
    matcher.add("TO_MERGE", None, *patterns)
    matches = matcher(doc)
    with doc.retokenize() as retokenizer:
        for match_id, start, end in matches:
            span = doc[start:end]
            retokenizer.merge(span)
    return doc

In [4]:
def split_word(file_dir):
    temp = ''
    with open(file_dir, "r") as i:
        for w in i:
            if (w == "\n"):
                continue
            else:
                temp += w
    return temp.splitlines()
    

In [5]:
file_dir = "/Users/tony/Desktop/TextRank/oxford/Oxford Dictionary of Computer Science (7 ed.).txt"
oxf_vocab = split_word(file_dir)


In [6]:
len(oxf_vocab)

6511

In [7]:
import re
temp = "We introduce the multiresolution recurrent neural network, which extends the\nsequence-to-sequence Framework to model natural language generation as two\nparallel discrete stochastic processes: a sequence of high-level coarse tokens,\nand a sequence of natural language tokens."

def contains_word(s, w):
    return (' ' + w + ' ') in (' ' + s + ' ')

'''
for i in range(len(oxf_vocab)):
    if contains_word(temp, oxf_vocab[i]):
        print(oxf_vocab[i])
'''

output_str = re.sub('[^A-Za-z0-9]+', ' ', temp)

contains_word(output_str, "neural network")


True

In [8]:
nlp.add_pipe(custom_retokenizer, before="tagger")

In [9]:
class TextRankForKeyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        
        '''
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                #print("each token: " + str(type(token)))
                #print("each token text: " + str(type(token.text)))
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            print(selected_words)
            sentences.append(selected_words)
        return sentences
        '''
        
        '''
        sentences = []
       
        for sent in doc.sents:
            selected_words = []
            print(sent.text)
            for token in sent:
                if token.text in oxf_vocab:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            print("selected_words: \n")
            print(selected_words)
            sentences.append(selected_words)   
        '''
        #for line in text.split('\n'):

        sentences = []
        a = 0
        for sent in doc.split('.'):
            if (a%5000 == 0):
                print("Processed sentences: " + str(a))

            a+=1

            selected_words = []
            text = sent.lower()
            token_text = word_tokenize(text)
            Pos_tag = nltk.pos_tag(token_text)
            lemmatized_text = self.lemmatize(Pos_tag)
            stopwords_plus = self.stopwords_list(Pos_tag)
            phrases = self.phrase_list(lemmatized_text, stopwords_plus)
            
            for i in phrases:
                temp = ""
                for j in i:
                    temp += (j + ' ')
                #print("temp: " + temp)
                for word in oxf_vocab:
                    if contains_word(temp, word):
                        selected_words.append(word)
            #print("selected_words: \n")
            #print(selected_words)
            sentences.append(selected_words)

        #print("sentences: \n")
        #print(sentences)
          
        return sentences
    
     
    

    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
    
    # formula to calculate the weights of nodes
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal()) 
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        d = {}
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            d[key] = value
            #print(d)
            if i > number:
                break
            
        return d

    def lemmatize(self, POS_tag):
        wordnet_lemmatizer = WordNetLemmatizer()
        adjective_tags = ['JJ','JJR','JJS']

        lemmatized_text = []

        for word in POS_tag:
            if word[1] in adjective_tags:
                lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0],pos="a")))
            else:
                lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0])))
        return lemmatized_text
    
    def stopwords_list(self, POS_tag):
        stopwords = []
        #POS_tag = nltk.pos_tag(lemmatized_text)POS_tag = nltk.pos_tag(lemmatized_text)
        wanted_POS = ['NN','NNS','NNP','NNPS','JJ','JJR','JJS','VBG','FW'] 

        for word in POS_tag:
            if word[1] not in wanted_POS:
                stopwords.append(word[0])

        punctuations = list(str(string.punctuation))

        stopwords = stopwords + punctuations

        stopword_file = open("long_stopwords.txt", "r")
        #Source = https://www.ranks.nl/stopwords

        lots_of_stopwords = []

        for line in stopword_file.readlines():
            lots_of_stopwords.append(str(line.strip()))

        stopwords_plus = []
        stopwords_plus = stopwords + lots_of_stopwords
        stopwords_plus = set(stopwords_plus)
        return stopwords_plus

    def phrase_list(self, lemmatized_text, stopwords_plus):
        phrases = []
        phrase = " "
        for word in lemmatized_text:
            
            if word in stopwords_plus:
                if phrase!= " ":
                    phrases.append(str(phrase).strip().split())
                phrase = " "
            elif word not in stopwords_plus:
                phrase+=str(word)
                phrase+=" "

        #print("Partitioned Phrases (Candidate Keyphrases): \n")
        #print(phrases)
        return phrases

    def analyze(self, text, 
                candidate_pos=['NOUN', 'ADJ'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        #self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        

        '''
        text = text.lower()
        token_text = word_tokenize(text)
        
        Pos_tag = nltk.pos_tag(token_text)
        lemmatized_text = self.lemmatize(Pos_tag)
        stopwords_plus = self.stopwords_list(Pos_tag)
        phrases = self.phrase_list(lemmatized_text, stopwords_plus)
        '''

        '''
        for sent in doc.sents:
            for token in sent:
                print(token.text)
        '''

        '''
        for sent in doc.sents:
            displacy.render(sent, style="dep")    
        for sent in doc.sents:
            displacy.render(sent, style="ent")
        '''

        # Filter sentences
        sentences = self.sentence_segment(text, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)

        # print(token_pairs)

        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initialization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight


In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
import json
with open("arxivData.json", "r") as read_file:
    data = json.load(read_file)

corpus = []
for i in range(len(data)):
    corpus.append(data[i]["summary"])


In [12]:
len(corpus)

41000

In [13]:
import json
with open("arxivData.json", "r") as read_file:
    data = json.load(read_file)

corpus = ""
for i in range(len(data)):
    corpus += data[i]["summary"]

In [14]:

text = '''
We introduce the multiresolution recurrent neural network, which extends the\nsequence-to-sequence framework to model natural language generation as two\nparallel discrete stochastic processes: a sequence of high-level coarse tokens,\nand a sequence of natural language tokens. There are many ways to estimate or\nlearn the high-level coarse tokens, but we argue that a simple extraction\nprocedure is sufficient to capture a wealth of high-level discourse semantics.\nSuch procedure allows training the multiresolution recurrent neural network by\nmaximizing the exact joint log-likelihood over both sequences. In contrast to\nthe standard log- likelihood objective w.r.t. natural language tokens (word\nperplexity), optimizing the joint log-likelihood biases the model towards\nmodeling high-level abstractions. We apply the proposed model to the task of\ndialogue response generation in two challenging domains: the Ubuntu technical\nsupport domain, and Twitter conversations. On Ubuntu, the model outperforms\ncompeting approaches by a substantial margin, achieving state-of-the-art\nresults according to both automatic evaluation metrics and a human evaluation\nstudy. On Twitter, the model appears to generate more relevant and on-topic\nresponses according to automatic evaluation metrics. Finally, our experiments\ndemonstrate that the proposed model is more adept at overcoming the sparsity of\nnatural language and is better able to capture long-term structure.

'''

tr4w = TextRankForKeyword()
tr4w.analyze(corpus, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)



Processed sentences: 0
Processed sentences: 5000
Processed sentences: 10000
Processed sentences: 15000
Processed sentences: 20000
Processed sentences: 25000
Processed sentences: 30000
Processed sentences: 35000
Processed sentences: 40000
Processed sentences: 45000
Processed sentences: 50000
Processed sentences: 55000
Processed sentences: 60000
Processed sentences: 65000
Processed sentences: 70000
Processed sentences: 75000
Processed sentences: 80000
Processed sentences: 85000
Processed sentences: 90000
Processed sentences: 95000
Processed sentences: 100000
Processed sentences: 105000
Processed sentences: 110000
Processed sentences: 115000
Processed sentences: 120000
Processed sentences: 125000
Processed sentences: 130000
Processed sentences: 135000
Processed sentences: 140000
Processed sentences: 145000
Processed sentences: 150000
Processed sentences: 155000
Processed sentences: 160000
Processed sentences: 165000
Processed sentences: 170000
Processed sentences: 175000
Processed sentenc

In [19]:
dict_rank = tr4w.get_keywords(500)

model - 13.125795771889857
system - 12.383422221706004
algorithm - 12.34557216883533
data - 11.759797269491274
network - 11.267566772090648
image - 9.857292531626232
learning - 9.659608261164037
set - 8.644454337444229
application - 8.286443898671838
task - 8.108037845394637
function - 7.918245903983683
framework - 7.820828290510847
language - 7.812459496466648
process - 7.533624741655914
structure - 7.4050061126595494
representation - 7.393812622414277
order - 7.193396733553941
neural network - 6.717603676881666
point - 6.68934553183287
term - 6.684352251587636
input - 6.626912635113543
machine - 6.566161351356564
type - 6.420493010447493
class - 6.41353335900859
parameter - 6.328547470326026
domain - 6.308883526128389
distribution - 6.186170018838922
object - 6.1582653523345545
pattern - 6.119195870922046
knowledge - 6.073562089584464
architecture - 6.0407433324245305
accuracy - 6.023945502174273
search - 6.003048961976546
form - 5.962380274010048
dynamic - 5.90426024064353
level - 5

In [20]:
len(dict_rank)

502

In [17]:
text1 = '''
We introduce the multiresolution recurrent neural network, which extends the\nsequence-to-sequence framework to model natural language generation as two\nparallel discrete stochastic processes: a sequence of high-level coarse tokens,\nand a sequence of natural language tokens. There are many ways to estimate or\nlearn the high-level coarse tokens, but we argue that a simple extraction\nprocedure is sufficient to capture a wealth of high-level discourse semantics.\nSuch procedure allows training the multiresolution recurrent neural network by\nmaximizing the exact joint log-likelihood over both sequences. In contrast to\nthe standard log- likelihood objective w.r.t. natural language tokens (word\nperplexity), optimizing the joint log-likelihood biases the model towards\nmodeling high-level abstractions. We apply the proposed model to the task of\ndialogue response generation in two challenging domains: the Ubuntu technical\nsupport domain, and Twitter conversations. On Ubuntu, the model outperforms\ncompeting approaches by a substantial margin, achieving state-of-the-art\nresults according to both automatic evaluation metrics and a human evaluation\nstudy. On Twitter, the model appears to generate more relevant and on-topic\nresponses according to automatic evaluation metrics. Finally, our experiments\ndemonstrate that the proposed model is more adept at overcoming the sparsity of\nnatural language and is better able to capture long-term structure.
'''


In [18]:
for line in text1.split('.'):
    print("line: " + line)

line: 
We introduce the multiresolution recurrent neural network, which extends the
sequence-to-sequence framework to model natural language generation as two
parallel discrete stochastic processes: a sequence of high-level coarse tokens,
and a sequence of natural language tokens
line:  There are many ways to estimate or
learn the high-level coarse tokens, but we argue that a simple extraction
procedure is sufficient to capture a wealth of high-level discourse semantics
line: 
Such procedure allows training the multiresolution recurrent neural network by
maximizing the exact joint log-likelihood over both sequences
line:  In contrast to
the standard log- likelihood objective w
line: r
line: t
line:  natural language tokens (word
perplexity), optimizing the joint log-likelihood biases the model towards
modeling high-level abstractions
line:  We apply the proposed model to the task of
dialogue response generation in two challenging domains: the Ubuntu technical
support domain, and Twitte